## Model training

In [5]:
import pandas as pd
df=pd.read_csv('./data/gemstone.csv')

In [6]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
x=df.drop(labels=['id'],axis=1)
y=df[['price']]

In [8]:
x


,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [9]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [10]:
## Segregate Numerical and Categarical Column
categarical_column=x.columns[x.dtypes=='object']
numerical_column=x.columns[x.dtypes!='object']


In [11]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [14]:
from sklearn.impute import SimpleImputer   ## handaling Missing Value
from sklearn.preprocessing import StandardScaler## Scaling
from sklearn.preprocessing import OrdinalEncoder   ## oridinal encoding
from sklearn.pipeline import Pipeline   ## creating pipeline
from sklearn.compose import ColumnTransformer   ## Combining To pipeline

In [16]:
num_pipeline=Pipeline(
    steps=[
       ('imputer',SimpleImputer(strategy='median')),
       ('scler',StandardScaler()) 
    ]
)

In [18]:
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('oridinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
    
)

In [19]:
preprocessor=ColumnTransformer(
    [
        
        
        ('num_pipeline',num_pipeline,numerical_column),
        ('cat_pipeline',cat_pipeline,categarical_column)
    ]
)

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)

In [22]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())

In [23]:
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [25]:
x_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,num_pipeline__price,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,-0.691954,4.0,1.0,5.0
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,0.851645,2.0,4.0,4.0
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,2.172502,3.0,4.0,3.0
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,-0.731800,4.0,2.0,6.0
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-0.874107,2.0,5.0,2.0


In [26]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [28]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [29]:
regression.coef_

array([[ 1.10531784e-11, -5.29237547e-13,  8.91023224e-13,
         1.67551986e-12, -7.10678693e-12, -6.40171819e-13,
         4.04055821e+03,  2.04031831e-13,  3.98546590e-14,
        -4.47572585e-13]])

In [30]:
regression.intercept_

array([3976.8787389])

In [31]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [33]:
## Train multiple models
## Model Ecaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make Predictions
    y_pred=model.predict(x_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 2.577379612768072e-12
MAE: 1.8038486471927262e-12
R2 score 100.0


Lasso
Model Training Performance
RMSE: 1.3059909418341313
MAE: 0.8287379887549707
R2 score 99.99998944475847


Ridge
Model Training Performance
RMSE: 0.11861030169660734
MAE: 0.07891990937843393
R2 score 99.99999991293718


Elasticnet
Model Training Performance
RMSE: 1022.3507022713933
MAE: 709.9291087021516
R2 score 93.53173736515757




In [34]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']